In [ ]:
(* Related MSE posts *)

(* https://mathematica.stackexchange.com/questions/56499/the-winding-number-for-the-circle-map-arnold-tongue *)
(* https://mathematica.stackexchange.com/questions/289201/plotting-arnold-tongues/289245#289245 *)

# SRC

In [ ]:
(* Set number of threads *)

SetSystemOptions["ParallelOptions" -> {"ParallelThreadNumber" -> 32}] ;

(* Set compiler options *)

Get["CCompilerDriver`"] ;
Compiler`$CCompilerOptions = {"SystemCompileOptions" -> "-O3 -ffast-math -march=native"} ;

(* Set compilation target (use "WVM" if no compiler is avaliable ) *)

ClearAll[target] ;
target = "C" ;

(* Winding number computation using weighted Birkhoff average *)

meshgrid[x_List, y_List] := Developer`ToPackedArray@{ConstantArray[x, Length[y]], Transpose[ConstantArray[y, Length[x]]]} ;

(* Analytical filter *)

ClearAll[filter] ;
filter[order_, size_][1]     := 0.0 ;
filter[order_, size_][size_] := 0.0 ;
filter[order_, size_][i_]    := N[$MachineEpsilon + Exp[-(1/((1 - (-1 + i)/(-1 + size))^order*((-1 + i)/(-1 + size))^order))]] ;

(* Winding number computation *)

ClearAll[winding$number$birkhoff];
winding$number$birkhoff = Compile[
    {
        {window, _Real, 1}, 
        {eta, _Real}, 
        {epsilon, _Real}, 
        {phi, _Real}
    },
   Block[{size, result = 0.0, this = phi, next},
        size = Length[window];
        Do[
            next = this + eta + epsilon * Sin[this];
            result += window[[i]] * (next - this) ;
            this = next,
            {i, 1, size}
        ] ;
        result/(2*Pi)
    ],
    RuntimeAttributes -> {Listable},
    Parallelization -> True,
    CompilationTarget -> target,
    CompilationOptions -> {"ExpressionOptimization" -> True, "InlineExternalDefinitions" -> True},
    RuntimeOptions -> "Speed"
] ;

In [ ]:
<< CompiledFunctionTools`

CompilePrint[winding$number$birkhoff]

# Fixed initial

In [ ]:
(* Set number of iterations and grid size *)

n = 10^3 ;
m = 2001 ;

(* Set filter *)

window = Quiet[filter[1.0, n] /@ Range[n]] ;
window = window/Total[window] ;

(* Generate grid *)

etas = 2*Pi*Subdivide[0.0, 0.5, m - 1] ;
epsilons = 2*Pi*Subdivide[0.0, 1.0, m - 1] ;
{etas, epsilons} = meshgrid[etas, epsilons] ;

Dimensions[etas]
Dimensions[epsilons]

(* Set initial *)

phi = 0.0 

In [ ]:
(* Compute *)

result = Reverse[winding$number$birkhoff[window, etas, epsilons, phi]] ; // AbsoluteTiming
Dimensions[result]

In [ ]:
(* Save *)

Export["fixed.mx", result] ;

In [ ]:
(* Plot *)

ArrayPlot[
 	result, 
 	Frame -> False, 
 	ImageSize -> 800, 
 	AspectRatio -> 1, 
 	PlotRangePadding -> None,
 	ColorFunction -> (Blend[{Black, Blue, Green, Yellow, Red}, #] &), 
 	ColorFunctionScaling -> False
 ]

# Averaged over random initials

In [ ]:
(* Set number of iterations and grid size *)

n = 10^3 ;
m = 2001 ;

(* Set filter *)

window = Quiet[filter[1.0, n] /@ Range[n]] ;
window = window/Total[window] ;

(* Generate grid *)

etas = 2*Pi*Subdivide[0.0, 0.5, m - 1] ;
epsilons = 2*Pi*Subdivide[0.0, 1.0, m - 1] ;
{etas, epsilons} = meshgrid[etas, epsilons] ;

(* Set random initials *)

phis := RandomVariate[UniformDistribution[{0.0, 2.0*Pi}], Dimensions[etas]] ;

Dimensions[etas]
Dimensions[epsilons]
Dimensions[phis]

In [ ]:
(* Compute *)

count = 1000 ;
result = ConstantArray[0.0, {m, m}] ;
Do[
    result += winding$number$birkhoff[window, etas, epsilons, phis],
    {count}
] ; // AbsoluteTiming
result = Reverse[result]/count ; 

In [ ]:
(* Save *)

Export["averaged.mx", result] ;

In [ ]:
(* Plot *)

ArrayPlot[
 	result, 
 	Frame -> False, 
 	ImageSize -> 800, 
 	AspectRatio -> 1, 
 	PlotRangePadding -> None,
 	ColorFunction -> (Blend[{Black, Blue, Green, Yellow, Red}, #] &), 
 	ColorFunctionScaling -> False
 ]